In [ ]:
from dotenv import load_dotenv
import os
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import (
    AssetType,
    BalanceAllowanceParams,
    MarketOrderArgs,
    OrderArgs,
    OrderType,
)
host: str = "https://clob.polymarket.com"
chain_id: int = 137
load_dotenv()
key = os.getenv("POLYGON_WALLET_PRIVATE_KEY")
funder = os.getenv("FUNDER")
client = ClobClient(
    host, key=key, chain_id=chain_id, signature_type=1, funder=funder
)  #
creds = client.create_or_derive_api_creds()
client.set_api_creds(creds)
balance = client.get_balance_allowance(
    params=BalanceAllowanceParams(asset_type=AssetType.COLLATERAL)
)
print(balance)


In [ ]:
from datetime import datetime, timezone
from dateutil import parser
from nba_api.stats.endpoints import  ScoreboardV2
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.static import teams

from agents.polymarket.gamma import GammaMarketClient as Gamma
import json

tag_slug = "cfb"
game_date = "2024-12-24"
gamma = Gamma()
querystring_params = {
    "limit": 1000,
    "active":True,
    "closed":False,
    "related_tags":True,
    "tag_slug":tag_slug
    # "slug": "nba-uta-por-2024-12-06"
    # "tag_id": "1,745,100639"
}
events = gamma.get_events(querystring_params=querystring_params)
print(f"len(events): {len(events)}")
with open("tmp.json", "w") as f:
    json.dump(events, f, indent=4)
filtered_events = []
for event in events:
    if "series" not in event:
        continue
    assert len(event["series"]) == 1
    if game_date in event["slug"]:
        filtered_events.append(event)
print(f"len(events): {len(filtered_events)}")
with open(f"assets/events_{tag_slug}_{game_date}.json", "w") as f:
    json.dump(filtered_events, f, indent=4)


In [ ]:
from datetime import datetime, timezone
from dateutil import parser
from nba_api.stats.endpoints import  ScoreboardV2
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.static import teams

from agents.polymarket.gamma import GammaMarketClient as Gamma
import json

slug = "will-bitcoin-hit-100k-today-12-20-24"
gamma = Gamma()
querystring_params = {
    "slug": slug
    # "tag_id": "1,745,100639"
}
events = gamma.get_events(querystring_params=querystring_params)
print(f"len(events): {len(events)}")
with open(f"assets/events_{slug}.json", "w") as f:
    json.dump(events, f, indent=4)

In [ ]:
# trades
from tools.utils import client
import json
import time
from py_clob_client.clob_types import ApiCreds, OrderScoringParams, TradeParams
time_now = int(time.time())
time_24h = time_now - 30*60*60
print(f"time_now: {time_now}, time_24h: {time_24h}")
params = TradeParams(
    before=time_now,
    after=time_24h,
)
trades = client.get_trades(params=params)
print(len(trades))
with open("scoring.json", "w") as f:
    f.write(json.dumps(trades))

In [ ]:
# market
from tools.utils import client

market = client.get_market(condition_id="0x5d899f726697759c9aefbc5201022857c360dec910c9b12d449cfd57c575458c")
print(market)

In [ ]:
# notifications
from tools.utils import client
import json
# collateral_address = client.get_collateral_address()
# exchange_address = client.get_exchange_address()
# conditional_address = client.get_conditional_address()
# address = client.get_address()
# print(f"collateral_address: {collateral_address}")
# print(f"exchange_address: {exchange_address}")
# print(f"conditional_address: {conditional_address}")
# print(f"address: {address}")
notifications = client.get_notifications()
print(notifications)

with open("notifications.json", "w") as f:
    f.write(json.dumps(notifications, indent=4))

In [ ]:
# price
from tools.utils import client, calculate_buy_market_price
token = "17331294850529461704424063677771092246911677503620864091153247176478669086283"
balance = 300
orderbook = client.get_order_book(token)
price = calculate_buy_market_price(orderbook, balance)
print(price)


In [ ]:
# polymarket
from agents.polymarket.polymarket import Polymarket
import os
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import ApiCreds, BalanceAllowanceParams, AssetType
from dotenv import load_dotenv

host: str = "https://clob.polymarket.com"
chain_id: int = 137
load_dotenv()
key = os.getenv("POLYGON_WALLET_PRIVATE_KEY")
funder = os.getenv("FUNDER")
client = ClobClient(host, key=key, chain_id=chain_id, signature_type=1, funder=funder)  #
creds = client.create_or_derive_api_creds()
client.set_api_creds(creds)
print(creds)
balance = client.get_balance_allowance( params=BalanceAllowanceParams(asset_type=AssetType.COLLATERAL))["balance"]
balance = float(balance) / 1e6
# load csv
print(f"balance: {balance}")

In [ ]:
# sell
from tools.utils import client, sell_with_market_price, get_team_token
from py_clob_client.clob_types import MarketOrderArgs, OrderType
from py_clob_client.order_builder.constants import SELL
# game_date = "2025-02-08"
# team_token = get_team_token(game_date, "nba")
# print(f"team_token: {team_token}")
sell_with_market_price("37195019279800556593829394936124589940378053324013574598094803830595267856452", 10)

In [ ]:
# order
from tools.utils import client
order_id = "0x01741c9e8a396f003ad4ed76c6268625699c40081ad732963363a71154d8bf34"
order = client.get_order(order_id)
print(order)

In [6]:
# buy in
from tools.utils import buy_in
token = "17331294850529461704424063677771092246911677503620864091153247176478669086283"
buy_price = 0.01
price_th = 0.
buy_balance = 1
buy_in(
    tokens=[token],
    buy_price=buy_price,
    price_threshold=price_th,
    buy_balance=buy_balance
)

2025-02-13 21:58:58,583 - utils.py[line:299] - INFO: tick_size is 0.01
2025-02-13 21:58:58,584 - utils.py[line:301] - INFO: Im buying 17331294850529461704424063677771092246911677503620864091153247176478669086283 at 0.01 for 100.0 shares
2025-02-13 21:58:59,051 - utils.py[line:314] - INFO: 17331294850529461704424063677771092246911677503620864091153247176478669086283 post_order res: {'errorMsg': '', 'orderID': '0x1b19b468aa094b5f915135c0a727a03c7d18092037d160c808c48c17ac205451', 'takingAmount': '', 'makingAmount': '', 'status': 'live', 'transactionsHashes': None, 'success': True}
2025-02-13 21:58:59,052 - utils.py[line:315] - INFO: 17331294850529461704424063677771092246911677503620864091153247176478669086283 order_book: OrderBookSummary(market='0x23ec92ce9262cfa4bfaf9bafb8d4696fa528ef2e8950b35300d5f6f41a95a399', asset_id='17331294850529461704424063677771092246911677503620864091153247176478669086283', timestamp='1739455137519', bids=[OrderSummary(price='0.01', size='57070'), OrderSummary(

(True, [0.01], 0.0)

In [ ]:
import time
from py_clob_client.clob_types import OrderArgs
from py_clob_client.order_builder.constants import BUY, SELL
from tools.utils import client
time_now = time.time()
# token = "6322414904707232355835340454691756227049555392823461747958280939481095637773"

# res = client.create_and_post_order(OrderArgs(token_id=token, side=SELL, price=0.999, size=9.527384595959596))
# print(res)
# orderid = "0x66c08ebbb6389f75d495fc36c92c811493c750aae3971082d60887ff546bfd88"
# time_now = time.time()
# while True:
#     order = client.get_order(orderid)
#     if order:
#         print(f"order: {order}")
#         break
# print(f"get order cost {time.time() - time_now}")
# tick_size = float(client.get_tick_size(token))
# print(f"get tick size cost {time.time() - time_now}")
# print(f"tick_size: {tick_size}")
# price = float(client.get_price(token, SELL)["price"])
# price = min(price, 1-tick_size)
# spread = float(client.get_spread(token)["spread"])

# print(f"time taken: {time.time() - time_now}")
# order_book = client.get_order_book(token)
# print(f"order_book: {order_book}")

In [ ]:
from tools.utils import get_team_token

tag_slug = "nfl"
game_date = "2024-12-08"
team_token = get_team_token(game_date=game_date, tag_slug=tag_slug)
print(f"team_token: {team_token}")


In [ ]:
234100 / 234100000

In [ ]:
import datetime
time_stamp = 1733548334
# to local time
local_time = datetime.datetime.fromtimestamp(time_stamp)
print(f"local_time: {local_time}")
# to utc time
utc_time = datetime.datetime.utcfromtimestamp(time_stamp)
print(f"utc_time: {utc_time}")
# to american ET time
et_time = datetime.datetime.fromtimestamp(time_stamp, tz=timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
print(f"et_time: {et_time}")

time_str = "2024-12-06 19:00:00"
stamp = datetime.datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S").timestamp()
print(f"stamp: {stamp}")

In [ ]:
from nba_api.stats.endpoints import  ScoreboardV2

from nba_api.stats.endpoints import playbyplayv2
from nba_api.stats.endpoints import boxscoretraditionalv2
play_by_play = playbyplayv2.PlayByPlayV2(game_id="0022400589")
print(play_by_play)
data = play_by_play.get_dict()  # 返回字典格式数据
df= play_by_play.get_data_frames()[0]  # 返回DataFrame格式数据
print(data.keys())
print(df.cols)
# game_date = "2024-11-25"
# board = ScoreboardV2(game_date=game_date)
# data_sets = board.data_sets
# for i, data_set in enumerate(data_sets):
#     df = data_set.get_data_frame()
#     if "GAME_ID" in df.columns and "HOME_TEAM_ID" in df.columns:
#         game_ids = df["GAME_ID"].values
#         for game_id in game_ids:
#             if "339" not in str(game_id):
#                 continue
#             # Query for games where the Celtics were playing
#             print(f"game_id: {game_id}")

#             # 获取比赛逐回合数据
#             play_by_play = playbyplayv2.PlayByPlayV2(game_id=game_id)
#             data = play_by_play.get_data_frames()[0]  # 返回DataFrame格式数据
#             data.to_csv(f"play_by_play_{game_id}.csv", index=False)
#             box_score =  boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
#             for data_set in box_score.data_sets:
#                 df = data_set.get_data_frame()
#                 df.to_csv(f"box_score_{game_id}.csv", index=False)


            

In [ ]:
import requests
import json

# 定义 GraphQL 查询
query = """
{
  orderFilledEvents(
    where: { makerAssetId: "84472013145774660380603252188679283514256956886017890103299178370217717606440" }
    orderBy: timestamp
    orderDirection: desc
    first:500
  ) {
    id
    transactionHash
    timestamp
    maker {
      id
    }
    taker {
      id
    }
    makerAmountFilled
    takerAmountFilled
    fee
  }
}
"""

# 设置 GraphQL 端点 URL
url = "https://api.goldsky.com/api/public/project_cl6mb8i9h0003e201j6li0diw/subgraphs/polymarket-orderbook-resync/prod/gn"

# 发送请求
response = requests.post(url, json={'query': query})

# 获取并打印响应数据
data = response.json()
print(len(data))
with open("order_filled_event.json", "w") as f:
    json.dump(data, f, indent=4)

In [ ]:
# check flip
from tools.utils import check_flip, get_time_played
import time
import logging
logging.basicConfig(level=logging.INFO)
time_str = "Q4 :10.7"
score_diff = 11
time_played = get_time_played(time_str)
print(f"time_played: {time_played}")
time_now = time.time()
flip_rate = check_flip(time_played, score_diff)
print(f"flip_rate: {flip_rate} time_taken: {time.time() - time_now}")

In [ ]:
import pandas as pd
from pathlib import Path
csv_path = Path("/home/zx/code/nba_api/assets/fromq3")
csv_files = list(csv_path.glob("*.csv"))
df = pd.concat([pd.read_csv(file) for file in csv_files])
game_ids = df["GAME_ID"].unique()
print(f"len(game_ids): {len(game_ids)}")
game_ids = df["GAME_ID"].values
print(f"len(game_ids): {len(game_ids)}")

In [ ]:
from tools.utils import df
import logging
logging.basicConfig(level=logging.INFO)
gamids = df["GAME_ID"].values
print(len(gamids))
print(len(set(gamids)))

In [ ]:
time_price = [
    (1.00011, 0.09999),
    (1.00011, 0.09998),
]
with open(f"assets/prices/t_t", "w") as f:
    for time_stamp, price in time_price:
        f.write(f"{time_stamp} {price:.6}\n")

In [ ]:
from datetime import datetime
import pytz

# UTC 时间
utc_time = "2024-12-22 18:00:00+00"

# 转换为 datetime 对象
utc_time_obj = datetime.fromisoformat(utc_time)

# 本地时区（自动根据系统时区）
local_time = utc_time_obj.astimezone()

print("本地时间:", local_time)


In [ ]:
import time
time.time()

In [ ]:
181.19+178.18*2